In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/Colab Notebooks/DATA586/anomaly-dection-project/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/Colab Notebooks/DATA586/anomaly-dection-project


In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import altair as alt
from wordcloud import WordCloud
from wordcloud import ImageColorGenerator
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('BGL_cleaned.csv')


In [4]:
df.head()

,id,time,unknown1,exact_time,unknown2,info,label
0,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.363779,R02-M1-N0-C:J12-U11,ras kernel info instruction cache parity error...,Normal
1,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.527847,R02-M1-N0-C:J12-U11,ras kernel info instruction cache parity error...,Normal
2,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.675872,R02-M1-N0-C:J12-U11,ras kernel info instruction cache parity error...,Normal
3,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.823719,R02-M1-N0-C:J12-U11,ras kernel info instruction cache parity error...,Normal
4,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03-15.42.50.982731,R02-M1-N0-C:J12-U11,ras kernel info instruction cache parity error...,Normal


# Model fitting

In [5]:
# # apply word of bags to the text data and apply logistic regression
# def vec_logistic_model(X, y):
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#   vectorizer = CountVectorizer()
#   train_count = vectorizer.fit_transform(X_train)
#   test_count = vectorizer.transform(X_test)
#   logReg = LogisticRegression()

#   logReg.fit(train_count, y_train)
#   pred_test = logReg.predict(test_count)
#   pred_prob = logReg.predict_proba(test_count)
#   print("Log loss on test:",metrics.log_loss(y_test, pred_prob))
#   print("Accuracy Score on validation:",metrics.accuracy_score(y_test, pred_test))

In [6]:
# X = df['info']
# y = df['label']

In [7]:
# vec_logistic_model(X,y)

In [8]:
vectorizer = CountVectorizer(max_features=200)
X_counts = vectorizer.fit_transform(df['info'])

In [9]:
bow_df = pd.DataFrame(
    X_counts.toarray(), columns=vectorizer.get_feature_names_out(), index=df["label"]
)
bow_df.head()

,ac,active,address,alerts,alignment,allreduce,app,apps,asic,assembly,...,tlb,torus,total,tree,type,urgent,vnm,volt,vpd,warning
label,,,,,,,,,,,,,,,,,,,,,
Normal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Normal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
count_df = bow_df.reset_index().groupby('label').sum()
count_df.head()

In [ ]:
plot_anomaly = count_df.T.reset_index().sort_values("Anomaly", ascending=False).iloc[0:40,]
plot_normal = count_df.T.reset_index().sort_values("Normal", ascending=False).iloc[0:40,]


In [ ]:
a = alt.Chart(plot_anomaly).mark_bar().encode(
    x=alt.X('index:N', sort='-y', title = 'key words'),
    y= alt.Y('Anomaly:Q',scale=alt.Scale(domain=[0, 220000]))
).properties(width=600,height=300)
n = alt.Chart(plot_normal).mark_bar().encode(
    x=alt.X('index:N', sort='-y',title = 'key words'),
    y= alt.Y('Normal:Q')
).properties(width=600,height=300)

a | n

In [ ]:
anomaly = df[df['label'] == 'Anomaly']
normal = df[df['label'] == 'Normal']


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16,15))
text = " ".join(str(i) for i in anomaly['info'])

wordcloud = WordCloud(background_color="white").generate(text)
axes[0].imshow(wordcloud, interpolation='bilinear')
axes[0].axis("off")
axes[0].set_title("Anomaly")

text = " ".join(str(i) for i in normal['info'])
wordcloud = WordCloud(background_color="white").generate(text)
axes[1].imshow(wordcloud, interpolation='bilinear')
axes[1].axis("off")
axes[1].set_title("Normal")
fig.tight_layout()